In [46]:
import pandas as pd
import numpy as np 
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from linearmodels.panel import PanelOLS

In [111]:
data = pd.read_csv(r'E:\COOLYEAH\smt_5\EKT\Ekonometrika-DataPanel\Data\Indonesian Salary Panel.csv')

data.head(10)

,REGION,YEAR,POVERTY INDEX (%),HUMAN DEVELOPMENT INDEX,POPULATION DENSITY (KM2),OPEN UNEMPLOYMENT RATE (%),HIGH SCHOOL EDUCATION LEVEL (%),GRDP (RP),SALARY (RP)
0,ACEH,2016,3.270,70.00,88,7.850,74.46,116374.299885,2118500
1,ACEH,2017,2.950,70.60,90,6.980,70.64,121240.978718,2500000
2,ACEH,2018,2.825,71.19,91,6.440,70.68,126824.365236,2700000
3,ACEH,2019,2.610,71.90,93,5.825,69.96,132069.620798,2916810
4,ACEH,2020,2.785,71.99,91,5.995,70.07,131580.967158,3165031
5,ACEH,2021,2.905,72.18,92,6.300,74.36,135274.039296,3165031
6,ACEH,2022,2.695,72.80,95,6.070,70.67,140971.715367,3166460
7,BALI,2016,0.520,73.65,727,2.005,73.65,137296.445217,1807600
8,BALI,2017,0.615,74.30,735,1.380,74.62,144933.312013,1956727
9,BALI,2018,0.605,74.77,743,1.140,78.67,154072.662607,2127157


In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REGION                           245 non-null    object 
 1   YEAR                             245 non-null    int64  
 2   POVERTY INDEX (%)                245 non-null    float64
 3   HUMAN DEVELOPMENT INDEX          245 non-null    float64
 4   POPULATION DENSITY (KM2)         245 non-null    int64  
 5   OPEN UNEMPLOYMENT RATE (%)       245 non-null    float64
 6   HIGH SCHOOL EDUCATION LEVEL (%)  245 non-null    float64
 7   GRDP (RP)                        238 non-null    float64
 8   SALARY (RP)                      245 non-null    int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 17.4+ KB


In [110]:
data.columns

Index(['REGION', 'YEAR', 'POVERTY INDEX (%)', 'HUMAN DEVELOPMENT INDEX',
       'POPULATION DENSITY (KM2)', 'OPEN UNEMPLOYMENT RATE (%)',
       'HIGH SCHOOL EDUCATION LEVEL (%)', 'GRDP (RP)', 'SALARY (RP)'],
      dtype='object')

#### Pre-Processing

In [79]:
print(data.isnull().sum())  # Melihat jumlah missing values di setiap kolom

REGION                             0
YEAR                               0
POVERTY INDEX (%)                  0
HUMAN DEVELOPMENT INDEX            0
POPULATION DENSITY (KM2)           0
OPEN UNEMPLOYMENT RATE (%)         0
HIGH SCHOOL EDUCATION LEVEL (%)    0
GRDP (RP)                          7
SALARY (RP)                        0
dtype: int64


In [80]:
print(data.columns)

Index(['REGION', 'YEAR', 'POVERTY INDEX (%)', 'HUMAN DEVELOPMENT INDEX',
       'POPULATION DENSITY (KM2)', 'OPEN UNEMPLOYMENT RATE (%)',
       'HIGH SCHOOL EDUCATION LEVEL (%)', 'GRDP (RP)', 'SALARY (RP)'],
      dtype='object')


In [81]:
data.fillna(data.mean(numeric_only=True), inplace=True)
  # Mengisi missing values dengan rata-rata

scaler = StandardScaler()'POVERTY INDEX (%)', 'HUMAN DEVELOPMENT INDEX', 'POPULATION DENSITY (KM2)', 
                    'OPEN UNEMPLOYMENT RATE (%)', 'HIGH SCHOOL EDUCATION LEVEL (%)', 'GRDP (RP)', 'SALARY (RP)'
columns_to_scale = []
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

In [112]:
# Menghapus baris dengan REGION == 'INDONESIA'
data = data[data['REGION'] != 'INDONESIA']

# Menampilkan daftar REGION untuk memastikan "INDONESIA" tidak ada
print(data['REGION'].unique())  

['ACEH' 'BALI' 'BANTEN' 'BENGKULU' 'DI YOGYAKARTA' 'DKI JAKARTA'
 'GORONTALO' 'JAMBI' 'JAWA BARAT' 'JAWA TENGAH' 'JAWA TIMUR'
 'KALIMANTAN BARAT' 'KALIMANTAN SELATAN' 'KALIMANTAN TENGAH'
 'KALIMANTAN TIMUR' 'KALIMANTAN UTARA' 'KEP. BANGKA BELITUNG' 'KEP. RIAU'
 'LAMPUNG' 'MALUKU' 'MALUKU UTARA' 'NUSA TENGGARA BARAT'
 'NUSA TENGGARA TIMUR' 'PAPUA' 'PAPUA BARAT' 'RIAU' 'SULAWESI BARAT'
 'SULAWESI SELATAN' 'SULAWESI TENGAH' 'SULAWESI TENGGARA' 'SULAWESI UTARA'
 'SUMATERA BARAT' 'SUMATERA SELATAN' 'SUMATERA UTARA']


In [114]:
print(data.isnull().sum())

REGION                             0
YEAR                               0
POVERTY INDEX (%)                  0
HUMAN DEVELOPMENT INDEX            0
POPULATION DENSITY (KM2)           0
OPEN UNEMPLOYMENT RATE (%)         0
HIGH SCHOOL EDUCATION LEVEL (%)    0
GRDP (RP)                          0
SALARY (RP)                        0
dtype: int64


In [85]:
# Definisikan variabel independen (X) dan dependen (y)
X = data[['POVERTY INDEX (%)', 'HUMAN DEVELOPMENT INDEX', 
        'POPULATION DENSITY (KM2)', 'OPEN UNEMPLOYMENT RATE (%)', 
        'HIGH SCHOOL EDUCATION LEVEL (%)', 'GRDP (RP)']]
y = data['SALARY (RP)']

# Tambahkan konstanta ke X
X = sm.add_constant(X)

# Model Pooled OLS
pooled_ols_model = sm.OLS(y, X).fit()

# Tampilkan hasil
print(pooled_ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:            SALARY (RP)   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     15.13
Date:                Fri, 29 Nov 2024   Prob (F-statistic):           1.34e-14
Time:                        23:13:25   Log-Likelihood:                -3462.5
No. Observations:                 238   AIC:                             6939.
Df Residuals:                     231   BIC:                             6963.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [61]:
# Model Fixed Effect (FE) dengan Entity Effects
model_fe = PanelOLS.from_formula('Q("SALARY (RP)") ~ Q("POVERTY INDEX (%)") + Q("HUMAN DEVELOPMENT INDEX") + Q("POPULATION DENSITY (KM2)") + Q("OPEN UNEMPLOYMENT RATE (%)") + Q("HIGH SCHOOL EDUCATION LEVEL (%)") + Q("GRDP (RP)") + EntityEffects', data=data)
res_fe = model_fe.fit()

# Summary
print(res_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:       Q('SALARY (RP)')   R-squared:                        0.7138
Estimator:                   PanelOLS   R-squared (Between):             -91.419
No. Observations:                 238   R-squared (Within):               0.7138
Date:                Fri, Nov 29 2024   R-squared (Overall):             -89.513
Time:                        22:54:37   Log-likelihood                   -3230.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      82.316
Entities:                          34   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(6,198)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             82.316
                            

In [87]:
# Residual Sum of Squares (RSS) untuk Common Effect
rss_ce = pooled_ols_model.ssr  # Sudah tersedia untuk model OLS

# Residual Sum of Squares (RSS) untuk Fixed Effect (Manual Calculation)
residuals_fe = model_fe.resids  # Residual dari model Fixed Effect
rss_fe = sum(residuals_fe**2)  # RSS adalah jumlah kuadrat residual

# Chow Test (Manual Calculation)
n = data.shape[0]  # Jumlah observasi
k = X.shape[1]     # Jumlah parameter
g = data.index.get_level_values('REGION').nunique()  # Jumlah grup (REGION)

numerator = (rss_ce - rss_fe) / (g - 1)
denominator = rss_fe / (n - g - k)
f_stat = numerator / denominator

# Tentukan p-value
from scipy.stats import f
p_value = 1 - f.cdf(f_stat, g - 1, n - g - k)

print(f"F-statistic: {f_stat}")
print(f"P-value: {p_value}")

F-statistic: 36.14681283127712
P-value: 1.1102230246251565e-16


In [ ]:
from linearmodels.panel import RandomEffects

# Random Effect Model
model_re = RandomEffects(y, X).fit()

# Estimasi parameter (beta) dan varians kovarians dari kedua model
beta_fe = model_fe.params  # Koefisien Fixed Effect
beta_re = model_re.params  # Koefisien Random Effect
cov_fe = model_fe.cov  # Varians-kovarians Fixed Effect
cov_re = model_re.cov  # Varians-kovarians Random Effect

# Hitung statistik Hausman
diff = beta_fe - beta_re
cov_diff = cov_fe - cov_re
hausman_stat = diff.T @ np.linalg.inv(cov_diff) @ diff
p_value_hausman = 1 - f.cdf(hausman_stat, len(diff), len(diff))

print(f"Hausman Test Statistic: {hausman_stat}")
print(f"P-value: {p_value_hausman}")

Hausman Test Statistic: -32.557730243309095
P-value: 1.0


In [90]:
# Random Effect Model (Final Estimation)
model_re = RandomEffects(y, X).fit()

# Tampilkan ringkasan hasil
print(model_re.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:            SALARY (RP)   R-squared:                        0.4111
Estimator:              RandomEffects   R-squared (Between):             -1.7966
No. Observations:                 238   R-squared (Within):               0.5867
Date:                Fri, Nov 29 2024   R-squared (Overall):             -0.9492
Time:                        23:31:12   Log-likelihood                   -3324.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      26.877
Entities:                          34   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(6,231)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             26.877
                            

In [97]:
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan

# Ambil residuals dari model yang sudah dilatih
residuals = model_fe.resids.values  # Konversi residual ke numpy array

# Gunakan variabel X yang sama seperti sebelumnya
# Misalnya, jika 'X' adalah DataFrame atau numpy array yang berisi variabel independen
exog_variables = X.values  # Pastikan ini adalah variabel independen yang sama digunakan di model

# Uji Breusch-Pagan untuk heteroskedastisitas
bp_test = het_breuschpagan(residuals, exog_variables)
bp_test_statistic, bp_test_p_value = bp_test[0], bp_test[1]

print(f"Breusch-Pagan Test Statistic: {bp_test_statistic}")
print(f"P-value: {bp_test_p_value}")

Breusch-Pagan Test Statistic: 9.196128491545556
P-value: 0.16284467940526554


In [107]:
# Model Fixed Effect (FE) dengan Entity 
model_fe = PanelOLS.from_formula('Q("SALARY (RP)") ~ Q("HUMAN DEVELOPMENT INDEX") + Q("POPULATION DENSITY (KM2)") + Q("GRDP (RP)") + EntityEffects', data=data)

res_fe = model_fe.fit()

# Summary
print(res_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:       Q('SALARY (RP)')   R-squared:                        0.7079
Estimator:                   PanelOLS   R-squared (Between):             -85.669
No. Observations:                 238   R-squared (Within):               0.7079
Date:                Fri, Nov 29 2024   R-squared (Overall):             -83.881
Time:                        23:52:12   Log-likelihood                   -3232.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      162.37
Entities:                          34   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(3,201)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             162.37
                            